In [ ]:
#libraries
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option("display.max_rows", None)
import plotly.express as px
import plotly
import plotly.offline
import plotly.graph_objs as go
import nbformat
#wrap text:
import textwrap
import random

In [ ]:
#read csv
concept = pd.read_csv('../datasets/CONCEPT.csv')
condition_occurrence = pd.read_csv('../datasets/CONDITION_OCCURRENCE.csv')
drug_exposure = pd.read_csv('../datasets/DRUG_EXPOSURE.csv')
person = pd.read_csv('../datasets/PERSON.csv')
hierarchy = pd.read_csv('../datasets/hierarchy.csv')
props = pd.read_csv('../datasets/hemonc_component_properties.csv')

In [ ]:
#list of valid drug categories from Ivy from RxNorm/HemOnc
sact=['Alkylating agent', 'Anti-CD38 antibody', 'Anti-CTLA-4 antibody', 'Anti-TACSTD2 antibody-drug conjugate', 'Anthracycline', 'Antiandrogen', 'Antifolate',
'Antimetabolite', 'Antitumor antibiotic', 'Anti-CD52 antibody', 'Anti-CD20 antibody', 'Anti-EGFR antibody', 'Anti-HER2 antibody', 'Anti-CD38 antibody', 'Anti-PD-1 antibody',
'Anti-PD-L1 antibody', 'Anti-RANKL antibody', 'Anti-SLAMF7 antibody','Anti-VEGF antibody', 'Aromatase inhibitor', 'Aromatase inhibitorsthird generation',
'Biosimilar', 'BRAF inhibitor', 'DNA methyltransferase inhibitor', 'Deoxycytidine analog', 'EGFR inhibitor', 'ERBB 2 inhibitor', 'Estrogen receptor inhibitor',
'Folic acid analog', 'Fluoropyrimidine', 'GnRH agonist', 'HDAC inhibitor', 'Human DNA synthesisinhibitor', 'Microtubule inhibitor', 'MTOR inhibitor',
'Nitrogen mustard', 'Nitrosourea', 'Neutral', 'PARP inhibitor', 'PARP1 inhibitor', 'PARP2 inhibitor', 'Phenothiazine', 'Platinum agent', 'Proteasome inhibitor',
'Purine analog', 'Pyrimidine analog', 'RANK ligand inhibitor', 'Selective estrogen receptor modulator', 'Somatostatin analog', 'T-cell activator',
'Targeted therapeutic', 'Taxane', 'Topoisomerase I inhibitor', 'Topoisomerase II inhibitor', 'Triazene', 'Vinca alkaloid', 'Xanthine oxidase inhibitor',
'WHO Essential Cancer Medicine']
#rxnorm = rxnorm[rxnorm['component_class_name'].isin(sact)]
props=props[props['component_class_name'].isin(sact)]
antican = props['concept_id_2']
drug_exposure=drug_exposure[drug_exposure['drug_concept_id'].isin(antican)]
#rxnorm['component_class_name'].value_counts()

In [ ]:
#concept lookup in concept df
concept_lookup = {c.concept_id: c.concept_name for c in concept.itertuples()}

In [ ]:
#add labels
def make_labels(df):
    for c in df.columns:
        if 'concept_id' in c:
            df[c.replace('_id', '_label')] = df[c].map(concept_lookup)
        if 'concept_id' in c or 'source' in c or len(df[df[c].notna()])==0:
            df = df.drop(c, axis=1)
    return df

In [ ]:
#label the following dfs
condition_occurrence_labelled = make_labels(condition_occurrence)
drug_exposure_labelled = make_labels(drug_exposure)
exclusions = ['dexamethasone']
drug_exposure_labelled=drug_exposure_labelled[~drug_exposure_labelled['drug_concept_label'].isin(exclusions)]

In [ ]:
'''if required, mask by a particular condition or set of conditions

# filter only by occurrences of Squamous cell carcinoma, NOS, of glottis
glottis = condition_occurrence[condition_occurrence.condition_concept_id==44500236]
# patient IDs matching this occurrence
glottis_patients = glottis.person_id.tolist()
# mask the drug exposures only by people matching the condition
mask = drug_exposure_labelled['person_id'].isin(glottis_patients)
masked = drug_exposure_labelled[mask]
'''

In [ ]:
#reduce DF down to relevant variables for the visualization
small = drug_exposure_labelled[['person_id', 'drug_exposure_start_datetime', 'drug_concept_label']]
small = small.dropna()
small = small.drop_duplicates()
small_sorted = small.sort_values('drug_concept_label')
small['drug_concept_label'] = small_sorted.groupby(['person_id', 'drug_exposure_start_datetime'])['drug_concept_label'].transform(lambda x : ' & '.join(x))
#small.head()
'''small['drug_concept_label'] = small['drug_concept_label'].str.replace('dexamethasone & cisplatin','cisplatin & dexamethasone')
small['drug_concept_label'] = small['drug_concept_label'].str.replace('dexamethasone & cetuximab','cetuximab & dexamethasone')
small['drug_concept_label'] = small['drug_concept_label'].str.replace('dexamethasone & carboplatin','carboplatin & dexamethasone')'''
small_nodup = small_sorted.drop_duplicates()
#small_nodup['drug_concept_label']=small_nodup['drug_concept_label'].str.replace('& ', '&<br>')

In [ ]:
# add new variable for every new drug administration per person
readministrations = pd.Series(np.zeros(len(small_nodup),dtype=int),index=small_nodup.index)

In [ ]:
# Loop through all unique ids                                                                                                                                                                                      
all_id = small_nodup['person_id'].unique()
id_administrations = {}
for pid in all_id:
    # These are all the times a patient with a given ID has had surgery                                                                                                                                            
    patient = small_nodup.loc[small_nodup['person_id']==pid]
    administrations_sorted = pd.to_datetime(patient['drug_exposure_start_datetime'], format='%Y-%m-%d %H:%M:%S').sort_values()

# This checks if the previous surgery was longer than 180 days ago                                                                                                                                              
    frequency = administrations_sorted.diff()<dt.timedelta(days=6000)

    # Compute the readmission                                                                                                                                                                                      
    n_administrations = [0]
    for v in frequency.values[1:]:
       n_administrations.append((n_administrations[-1]+1)*v)

    # Add these value to the time series                                                                                                                                                                           
    readministrations.loc[administrations_sorted.index] = n_administrations

small_nodup['readministration'] = readministrations

In [ ]:
#pivot the DF from long to wide
pivoted = small_nodup.pivot(index='person_id', columns='readministration', values='drug_concept_label').reset_index()
# add the prefix 'drug' to every instance
prefixed = pivoted.add_prefix('drug')
#remove the word 'drug' from other variables
renamed = prefixed.rename(columns={"drugperson_id": "person_id", "readministration":"index"})
#fill all empty cells with "N/A"
#fillednones = renamed.fillna(" ")
fillednones = renamed

In [ ]:
#add a value of 1 to all data points for sums in the visualization
fillednones["count"] = 1
fillednones.head()

In [ ]:
df = fillednones

In [ ]:
def genSankey(df, cat_cols=[], value_cols='', title='Sankey Diagram'):
    #color palette
    #skip
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp = list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
    
    #remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))

    #define colors based on number of levels
    #skip

    #transform df into asource-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i], cat_cols[i+1], value_cols]]
            sourceTargetDf.columns = ['source', 'target', 'count']
        else:
            tempDf = df[[cat_cols[i], cat_cols[i+1], value_cols]]
            tempDf.columns = ['source', 'target', 'count']
            sourceTargetDf = pd.concat([sourceTargetDf, tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
    
    #add index for source-target pairs
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x:labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x:labelList.index(x))

    nodes = np.unique(sourceTargetDf[["sourceID", "targetID"]], axis=None)
    nodes = pd.Series(index=nodes, data=range(len(nodes)))

    fig = go.Figure(
        go.Sankey(
            node = {
                "label": labelList,
                "color":[
                    px.colors.qualitative.Set1[i % 9]
                    for i in nodes
                ]
            },
            link = {
                "source": sourceTargetDf["sourceID"],
                "target": sourceTargetDf["targetID"],
                "value": sourceTargetDf["count"],
                "color": [
                    px.colors.qualitative.Pastel1[i % 9]
                    for i in nodes.loc[sourceTargetDf["sourceID"]]
                ],
            },
        )
    )


    return fig

'''
    #draw the sankey diagram
    data= dict(
        type='sankey',
        node= dict(
            pad = 15,
            thickness = 20,
            line = dict(
                color = "black",
                width = 0.5
            ),
            label = labelList
            #color = colorList
        ),
        link = dict(
            source = sourceTargetDf['sourceID'],
            target = sourceTargetDf['targetID'],
            value= sourceTargetDf['count']
        )
    )
    layout = dict(
        title= title,
        font = dict(
            size = 12
        )
    )
    
    fig = dict(data=[data], layout=layout)
'''

In [ ]:
#run the sankey function
fig = genSankey(df, cat_cols=['drug0','drug1','drug2','drug3','drug4','drug5','drug6','drug7','drug8','drug9','drug10','drug11','drug12','drug13','drug14','drug15','drug16','drug17','drug18','drug19'], value_cols='count', title='Sankey Diagram')
fig.update_layout(margin = dict(l=0, r=0, t=0, b=0))
plotly.offline.plot(fig, validate=False)